In [10]:
import numpy as np
from random import randrange
import re
from statistics import stdev

In [11]:
filename='Stopwords'

def Stopwords(filename):
    file1 = open(filename, 'r') 
    #print(file1)
    Lines = file1.read() #List of strings
    #print(Lines)  
    Stopwords=Lines.split('\n')
    #print(Stopwords)
    
    return Stopwords

#Stopwords(filename)

In [12]:
### remove unnecessary functions/code
### add laplacian for improvement
### preprocessing
### load stopwords from somewhere

def split_lines(filename):
    file1 = open(filename, 'r') 
    Lines = file1.readlines() #List of strings  
    lines_list=[]
    # Strips the newline character 
    for line in Lines: 
        lst=[]        
        lst.append(line.strip().split("\t")[0])#sentence 
        lst.append(int(line.strip().split("\t")[1]))#class label
        lines_list.append(lst)
        #print(lines_list) #[[sdsad,1], [sasda,0]]
    return lines_list

def tokenizer(string):

    string=re.sub('[^A-Za-z0-9]+', ' ', string) # Removing Special Symbols    
    string=string.lower() # Converting the txt to lowercase   
    stopWords = set(Stopwords(filename))
    string=" ".join(word for word in string.split() if word not in stopWords) #Removing stop words      
    return string.split()

def compute_counts(X,Y):
    dictionary={}#Empty Dictionary [+ve,-ve]
    for i in range(0,len(X)):
        for word in X[i]:
            if word not in dictionary:
                dictionary[word]=[1,1] # laplacian smoothing(Min occurrence is 1)
            if Y[i]==0:
                dictionary[word][0]+=1
            else:
                dictionary[word][1]+=1
    return dictionary

def compute_label_count(dictionary):
    total_neg_labels=0#0
    total_pos_labels=0#1
    for word in dictionary:
        total_neg_labels+=dictionary[word][0]
        total_pos_labels+=dictionary[word][1]

    return total_neg_labels,total_pos_labels

def predict(sentence,dictionary,neg_count,pos_count,neg_pro,pos_pro):
    #neg_prob=neg_count/(neg_count+pos_count)#Initialise with prior probability P(Y=0)
    #pos_prob=pos_count/(neg_count+pos_count)#Initialise with prior probability P(Y=1)
    neg_prob=neg_pro
    pos_prob=pos_pro
    #calcultate -ve probablity (numerator only as denominator is same for both the classes)
    if(neg_prob!=0):
        for word in sentence:
            if word not in dictionary:
                pass#need not handle
            else:
                if neg_prob==None:
                    neg_prob=dictionary[word][0]/neg_count #P(xi|Y=0)
                else:
                    neg_prob*=dictionary[word][0]/neg_count
    
    #calcultate +ve probablity (numerator only as denominator is same for both the classes)
    if(pos_prob!=0):
        for word in sentence:
            if word not in dictionary:
                pass#need not handle out of vocabulary words
            else:
                if pos_prob==None:
                    pos_prob=dictionary[word][1]/pos_count #P(xi|Y=0)
                else:
                    pos_prob*=dictionary[word][1]/pos_count

    #print("Positive Prob(numerator):",pos_prob)
    #print("Negative Prob(numerator):",neg_prob)
    if neg_prob >= pos_prob:
        return 0
    else:        
        return 1

# Split a dataset into k folds
def cross_validation_split(X,Y, n_folds):
    X_split = list()
    Y_split = list()
    dataset_copy_X = list(X)
    dataset_copy_Y = list(Y)
    fold_size = int(len(X) / n_folds)
    for i in range(n_folds):
        fold_X = list()
        fold_Y = list()
        while len(fold_X) < fold_size:
            index = randrange(len(dataset_copy_X))
            #print(index)
            fold_X.append(dataset_copy_X.pop(index))
            fold_Y.append(dataset_copy_Y.pop(index))
        X_split.append(fold_X)
        Y_split.append(fold_Y)
    return X_split,Y_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(X,Y, algorithm, n_folds):
    X_folds,Y_folds = cross_validation_split(X,Y, n_folds)
    accuracies = list()

    for (fold_X,fold_Y) in zip(X_folds,Y_folds):
        train_set_X = list(X_folds)
        train_set_Y = list(Y_folds)
        train_set_X.remove(fold_X)
        train_set_Y.remove(fold_Y)
        train_set_X = sum(train_set_X, [])
        train_set_Y = sum(train_set_Y, [])
        test_set_X = list()
        test_set_Y = list()
        for (row_X,row_Y) in zip(fold_X,fold_Y):
            row_copy_X = list(row_X) 
            row_copy_Y = row_Y
            test_set_X.append(row_copy_X)
            test_set_Y.append(row_copy_Y)
          
        predicted = algorithm(train_set_X,train_set_Y, test_set_X,test_set_Y)
        actual = [num for num in fold_Y]
        accuracy = accuracy_metric(actual, predicted)
        accuracies.append(accuracy)

    return accuracies

def naive_bayes(X,Y,test_X,test_Y):
    dictionary=compute_counts(X,Y)
    
    neg_count,pos_count=compute_label_count(dictionary)#redundant simply use Y to get the counts later
   
    neg_prob=Y.count(0)/len(Y)#P(Y=0)
    pos_prob=Y.count(1)/len(Y)#P(Y=1)
    predictions=[]
    for i in range(0,len(test_X)):
        pred=predict(test_X[i],dictionary,neg_count,pos_count,neg_prob,pos_prob)
        predictions.append(pred)
    return predictions    

#start from here       
if __name__ == "__main__":
    row=split_lines('dataset_NB.txt')#get a list of lists with sentence and class label in each child list
    X=[]        
    '''
        X is a list of list of tokens like this:
        [['This','is','token'],
        ['Another','token']]
        
    '''
    for i in range(0,len(row)):
        tokens=tokenizer(row[i][0])#list of token strings
        # Removing symbols        
        X.append(tokens)
    
    Y=[ row[i][1] for i in range(0,len(row)) ]
    
    n_folds=7
    accuarcies =evaluate_algorithm(X,Y,naive_bayes,n_folds)
    print('Accuracies over each fold: %s' % accuarcies)
    print('Average Accuracy: %.3f' % (sum(accuarcies)/float(len(accuarcies))))

    '''
        Y is list of class labels 0 or 1 like this:
        [1,0,0,1,0,1,0]
       
    '''
    #dictionary=compute_counts(X,Y)
    
    '''
    dictonary contains all the words occuring in the text file and the count of occurences in -ve and +ve sentiments like this
    
    { '
       'This': [0, 1], # each index contains -ve and +ve occurences respectively
       'is': [1, 1],
       'good.': [0, 1],
       'It': [1, 0],
       'BAD': [2, 0]
    }
    
    '''
    #print(type(dictonary['This'][0]))
    #neg_count,pos_count=compute_label_count(dictionary) #gives total number of positive and negative samples in the dataset
    
    
    #predict method takes a sentence and predicts the class it might belong to
    
    #print(dictonary)
    #result=predict(['This', 'is', 'a' ,'test'],dictonary,neg_count,pos_count)#predict method takes a single sample and returns expected outcome
    #print("Predicted Outcome: ",result)

Accuracies over each fold: [76.76056338028168, 76.76056338028168, 78.16901408450704, 81.69014084507043, 81.69014084507043, 85.2112676056338, 76.76056338028168]
Average Accuracy: 79.577
